In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy import sparse as sp
from scipy.sparse import linalg as la
import time

**Задание матрицы оператора L**

In [3]:
def L_lil(n_x, n_y, h_x, h_y):
    k = np.arange(n_x * n_y, dtype=int).reshape((n_x, n_y))
    L = sp.lil_matrix((n_x * n_y, n_x * n_y))


    for i in range(n_x):
        for j in range(n_y):
            L[k[i, j], k[i, j]] = 2.0 / h_x ** 2 + 2.0 / h_y ** 2

            if i == 0:
                L[k[i, j], k[i, j]] += 1.0 / h_x ** 2
                L[k[i, j], k[i + 1, j]] = -1.0 / h_x ** 2
            elif i == n_x - 1:
                L[k[i, j], k[i, j]] += 1.0 / h_x ** 2
                L[k[i, j], k[i - 1, j]] = -1.0 / h_x ** 2
            else:
                L[k[i, j], k[i - 1, j]] = -1.0 / h_x ** 2
                L[k[i, j], k[i + 1, j]] = -1.0 / h_x ** 2

            if j == 0:
                L[k[i, j], k[i, j]] += 1.0 / h_y ** 2
                L[k[i, j], k[i, j + 1]] = -1.0 / h_y ** 2
            elif j == n_y - 1:
                L[k[i, j], k[i, j]] += 1.0 / h_y ** 2
                L[k[i, j], k[i, j - 1]] = -1.0 / h_y ** 2
            else:
                L[k[i, j], k[i, j - 1]] = -1.0 / h_y ** 2
                L[k[i, j], k[i, j + 1]] = -1.0 / h_y ** 2

    return L.tocsr()

In [4]:
def L_csr(n_x, n_y, h_x, h_y):
    k = np.arange(n_x * n_y, dtype=int).reshape((n_x, n_y))

    iB = k[:-1, :].flatten() 
    iT = k[1:, ].flatten() 

    iNB = k[-1, :].flatten() 
    iNT = k[0, :].flatten() 

    iR = k[:, :-1].flatten() 
    iL = k[:, 1:].flatten() 

    iNR = k[:, -1].flatten() 
    iNL = k[:, 0].flatten() 

    iC = np.arange(n_x * n_y)

    R1, C1, V1 = iC, iC, np.full(iC.size, 2 / h_x ** 2 + 2 / h_y ** 2)

    R2, C2, V2 = iL, iR, np.full(iL.size, -1.0 / h_x ** 2)
    R3, C3, V3 = iR, iL, np.full(iR.size, -1.0 / h_x ** 2)

    R4, C4, V4 = iB, iT, np.full(iB.size, -1.0 / h_y ** 2)
    R5, C5, V5 = iT, iB, np.full(iT.size, -1.0 / h_y ** 2)

    R6, C6, V6 = iNR, iNR, np.full(iNR.size, 1.0 / h_x ** 2)
    R7, C7, V7 = iNL, iNL, np.full(iNL.size, 1.0 / h_x ** 2)

    R8, C8, V8 = iNB, iNB, np.full(iNB.size, 1.0 / h_y ** 2)
    R9, C9, V9 = iNT, iNT, np.full(iNT.size, 1.0 / h_y ** 2)

    row = np.concatenate((R1, R2, R3, R4, R5, R6, R7, R8, R9))
    col = np.concatenate((C1, C2, C3, C4, C5, C6, C7, C8, C9))
    val = np.concatenate((V1, V2, V3, V4, V5, V6, V7, V8, V9))

    return sp.csr_matrix((val, (row, col)))

In [5]:
L1 = L_lil(6, 6, 1, 1)
L2 = L_csr(6, 6, 1, 1)
L1_array = L1.toarray()
L2_array = L2.toarray()

colorscale = [
    [0, '#727094'],
    [0.2, '#ffffff'],
    [0.7, '#99ccff'],
    [0.8, '#89d689'],
    [1, '#f5ad64']
]

fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=L1_array,
    colorscale=colorscale,
    colorbar=dict(title='Цветовая шкала', tickvals=[-1, 0, 4, 5, 6]),
    zmin=-1.5,
    zmax=6.5,
    name='LIL-матрица'
))

fig.add_trace(go.Heatmap(
    z=L2_array,
    colorscale=colorscale,
    colorbar=dict(title='Цветовая шкала', tickvals=[-1, 0, 4, 5, 6]),
    zmin=-1.5,
    zmax=6.5,
    name='CSR-матрица',
    xaxis='x2',
    yaxis='y2'
))

fig.update_layout(
    title='Разреженные матрицы LIL и CSR',
    grid=dict(rows=1, columns=2),
    width=1150,
    height=600,
    yaxis=dict(showgrid=False, zeroline=False, autorange='reversed'),
    yaxis2=dict(showgrid=False, zeroline=False, autorange='reversed'),
)

fig.show()

**Решение задачи из пункта а.)**

In [6]:
h_0 = 0.01
l_x = 1
l_y = np.sqrt(2)

n_x = int(np.round(l_x / h_0))
n_y = int(np.round(l_y / h_0))

h_x = l_x / n_x
h_y = l_y / n_y

x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

X, Y = np.meshgrid(x, y, indexing='ij')

In [7]:
def f(x, y):
  return np.sin(5 * x) * ((25 + np.pi ** 2 * y ** 2) * np.cos(np.pi * y ** 2 / 2) + np.pi * np.sin(np.pi * y ** 2 / 2))


def μ_L(y):
  return 0


def μ_R(y):
  return np.sin(5) * np.cos(np.pi * y ** 2 / 2)


def μ_B(x):
  return np.sin(5 * x)


def μ_T(x):
  return -np.sin(5 * x)


def u_0(x, y):
  return np.sin(5 * x) * np.cos(np.pi * y ** 2 / 2)

In [8]:
def RHS(X, Y, h_x, h_y):
    res = f(X, Y)

    # i = 1
    res[0, :] += 2 * μ_L(Y[0, :]) / h_x ** 2

    # i = n_x
    res[-1, :] += 2 * μ_R(Y[-1, :]) / h_x ** 2

    # j = 1
    res[:, 0] += 2 * μ_B(X[:, 0]) / h_y ** 2

    # j = n_y
    res[:, -1] += 2 * μ_T(X[:, -1]) / h_y ** 2

    return res

In [9]:
L = L_csr(n_x, n_y, h_x, h_y)
F = RHS(X, Y, h_x, h_y).flatten()

In [10]:
results = {
    'method': [],
    'time': [],
    'error': [],
}

# прямой решатель
start_time = time.time()
u_sp = la.spsolve(L, F)
end_time = time.time()
time_sp = end_time - start_time
results['method'].append('spsolve')
results['time'].append(time_sp)
results['error'].append(0)  # прямой решатель не имеет погрешности в этом контексте

# приближенные решатели
methods = ['bicg', 'bicgstab', 'cg', 'cgs', 'gmres', 'qmr']
for method in methods:
    start_time = time.time()
    solution, info = getattr(la, method)(L, F)
    end_time = time.time()
    time_method = end_time - start_time
    error = np.linalg.norm(u_sp - solution)  # погрешность по сравнению с прямым решателем

    results['method'].append(method)
    results['time'].append(time_method)
    results['error'].append(error)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=results['method'],
    y=results['time'],
    name='Время выполнения (с)',
    marker_color='#99ccff'
))

fig.add_trace(go.Scatter(
    x=results['method'],
    y=results['error'],
    mode='lines+markers',
    name='Погрешность',
    marker_color='black'
))

fig.update_layout(
    title='Сравнение решателей СЛАУ',
    xaxis_title='Метод',
    yaxis_title='Значение',
    yaxis=dict(title='Время (с)', side='left', showgrid=False),
    yaxis2=dict(title='Погрешность', overlaying='y', side='right', showgrid=False),
    legend=dict(x=0.1, y=1.1),
)

fig.show()

In [11]:
u_exact = u_0(X, Y)
u_num = la.spsolve(L, F).reshape((n_x, n_y))

In [12]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Приближенное решение", "Аналитическое решение"])

fig.add_trace(go.Heatmap(z=u_num.T, colorscale='ice', x0=0, dx=l_x, y0=0, dy=l_y, colorbar=dict(title='u')), row=1, col=1)
fig.add_trace(go.Heatmap(z=u_exact.T, colorscale='ice', x0=0, dx=l_x, y0=0, dy=l_y, colorbar=dict(title='u')), row=1, col=2)

fig.update_layout(
    width=1000,
    height=500,
)

fig.update_xaxes(title_text='x', row=1, col=1)
fig.update_xaxes(title_text='x', row=1, col=2)
fig.update_yaxes(title_text='y', row=1, col=1)
fig.update_yaxes(title_text='y', row=1, col=2)

fig.show()

**Вычисление погрешностей и указание порядка аппроксимации**

In [13]:
h_0 = 0.1

h = []
fault_array = []

for i in range(4):
    n_x = int(np.round(l_x / h_0))
    n_y = int(np.round(l_y / h_0))

    h_x = l_x / n_x
    h_y = l_y / n_y

    x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
    y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

    X, Y = np.meshgrid(x, y, indexing='ij')

    L = L_csr(n_x, n_y, h_x, h_y)
    F = RHS(X, Y, h_x, h_y).flatten()

    u_exact = u_0(X, Y)
    u_num = la.spsolve(L, F).reshape((n_x, n_y))

    fault_array.append(np.max(np.abs(u_exact - u_num)))

    h.append(h_0)

    h_0 /= 4

In [14]:
tg = abs(np.log(fault_array[2]) - np.log(fault_array[3])) / abs(np.log(h[2]) - np.log(h[3]))
print(tg)

fig = go.Figure(data=[go.Scatter(x=h, y=fault_array, mode='lines+markers', line=dict(color='black'), marker=dict(color='black'))])

fig.update_layout(title='Оценка точности метода',
                  xaxis_title='h',
                  yaxis_title='max_err',
                  xaxis=dict(type='log'),
                  yaxis=dict(type='log'))

fig.show()

1.7585537968900704
